<a href="https://colab.research.google.com/github/NethmiAmasha/Edge-Detection-with-Mamba/blob/main/Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check GPU and Install Basic Packages


In [1]:
print("=" * 70)
print("STEP 1: Checking your environment")
print("=" * 70)

import sys
print(f"Python version: {sys.version}")

# Check if GPU is available
import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️  WARNING: No GPU found. Training will be VERY slow.")
    print("   In Colab: Runtime → Change runtime type → GPU")

print("\n✓ Basic check complete!")

STEP 1: Checking your environment
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]

PyTorch version: 2.8.0+cu126
CUDA available: False
⚠️  WARNING: No GPU found. Training will be VERY slow.
   In Colab: Runtime → Change runtime type → GPU

✓ Basic check complete!


Install Required Packages

In [2]:
print("\n" + "=" * 70)
print("STEP 2: Installing required packages (this may take 2-3 minutes)")
print("=" * 70)

# Install packages quietly
import subprocess
import sys

def install_package(package):
    """Helper function to install packages quietly"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

packages = [
    "opencv-python",
    "scikit-image",
    "matplotlib",
    "timm",  # PyTorch Image Models
    "einops",  # For tensor operations
    "scipy",
]

print("\nInstalling packages...")
for pkg in packages:
    try:
        install_package(pkg)
        print(f"  ✓ {pkg}")
    except Exception as e:
        print(f"  ✗ {pkg} - Error: {e}")

print("\n✓ Package installation complete!")


STEP 2: Installing required packages (this may take 2-3 minutes)

Installing packages...
  ✓ opencv-python
  ✓ scikit-image
  ✓ matplotlib
  ✓ timm
  ✓ einops
  ✓ scipy

✓ Package installation complete!


Install Mamba (Special Installation)

In [3]:
print("\n" + "=" * 70)
print("STEP 3: Installing Mamba and dependencies")
print("=" * 70)
print("This is the most important part - Mamba requires special installation")
print("This might take 3-5 minutes...\n")

# Install causal-conv1d first (required for Mamba)
print("Installing causal-conv1d...")
try:
    subprocess.check_call([
        sys.executable, "-m", "pip", "install",
        "causal-conv1d>=1.1.0", "-q"
    ])
    print("  ✓ causal-conv1d installed")
except Exception as e:
    print(f"  ✗ Error: {e}")

# Install mamba-ssm
print("\nInstalling mamba-ssm...")
try:
    subprocess.check_call([
        sys.executable, "-m", "pip", "install",
        "mamba-ssm", "-q"
    ])
    print("  ✓ mamba-ssm installed")
except Exception as e:
    print(f"  ✗ Error: {e}")
    print("  Note: Mamba installation can be tricky. If it fails, we'll use alternatives.")

print("\n✓ Mamba installation complete!")


STEP 3: Installing Mamba and dependencies
This is the most important part - Mamba requires special installation
This might take 3-5 minutes...

Installing causal-conv1d...


KeyboardInterrupt: 

Test Imports

In [ ]:
print("\n" + "=" * 70)
print("STEP 4: Testing if everything works")
print("=" * 70)

test_results = {}

# Test basic imports
print("\nTesting imports...")

try:
    import torch
    import torchvision
    test_results['torch'] = '✓'
    print(f"  ✓ PyTorch {torch.__version__}")
except:
    test_results['torch'] = '✗'
    print(f"  ✗ PyTorch failed")

try:
    import cv2
    test_results['opencv'] = '✓'
    print(f"  ✓ OpenCV {cv2.__version__}")
except:
    test_results['opencv'] = '✗'
    print(f"  ✗ OpenCV failed")

try:
    import matplotlib.pyplot as plt
    test_results['matplotlib'] = '✓'
    print(f"  ✓ Matplotlib")
except:
    test_results['matplotlib'] = '✗'
    print(f"  ✗ Matplotlib failed")

try:
    from skimage import io
    test_results['skimage'] = '✓'
    print(f"  ✓ Scikit-image")
except:
    test_results['skimage'] = '✗'
    print(f"  ✗ Scikit-image failed")

try:
    import timm
    test_results['timm'] = '✓'
    print(f"  ✓ TIMM {timm.__version__}")
except:
    test_results['timm'] = '✗'
    print(f"  ✗ TIMM failed")

try:
    from mamba_ssm import Mamba
    test_results['mamba'] = '✓'
    print(f"  ✓ Mamba SSM")
except:
    test_results['mamba'] = '✗'
    print(f"  ✗ Mamba SSM failed (this might need troubleshooting)")

# Summary
print("\n" + "=" * 70)
print("INSTALLATION SUMMARY")
print("=" * 70)
failures = [k for k, v in test_results.items() if v == '✗']
if not failures:
    print("🎉 ALL PACKAGES INSTALLED SUCCESSFULLY!")
    print("\nYou're ready to proceed to data loading!")
else:
    print(f"⚠️  Some packages failed: {', '.join(failures)}")
    print("\nDon't worry! We can work around this.")

print("\n" + "=" * 70)

Create Directory Structure

In [ ]:
print("\n" + "=" * 70)
print("STEP 5: Creating project directories")
print("=" * 70)

import os

# Create directories
directories = [
    'data',
    'data/BSDS500',
    'data/BSDS500/train',
    'data/BSDS500/train/images',
    'data/BSDS500/train/edges',
    'data/BSDS500/test',
    'data/BSDS500/test/images',
    'data/BSDS500/test/edges',
    'models',
    'results',
    'checkpoints'
]

print("\nCreating directories...")
for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"  ✓ {directory}")

print("\n✓ Directory structure created!")
print("\nYour project structure:")
print("""
📁 Project Root
├── 📁 data/
│   └── 📁 BSDS500/
│       ├── 📁 train/
│       │   ├── 📁 images/
│       │   └── 📁 edges/
│       └── 📁 test/
│           ├── 📁 images/
│           └── 📁 edges/
├── 📁 models/
├── 📁 results/
└── 📁 checkpoints/
""")

print("=" * 70)
print("🎉 SETUP COMPLETE!")
print("=" * 70)
print("\nNext steps:")
print("1. Download BSDS500 dataset")
print("2. Explore the data")
print("3. Understand data loading")
print("\nReady to continue!")